In [ ]:
import numpy as np                
import matplotlib.pyplot as plt   
from matplotlib.animation import FuncAnimation  
from IPython.display import HTML, display 

#============================================================================
#Etape1: Configuration de l'environnement
#============================================================================


#Cette commande est spécifique à Jupyter Notebook. Elle permet d'afficher
#Les graphiques directement dans le notebook plutôt que dans une fenêtre séparée.
#Si vous utilisez un autre environnement (comme PyCharm ou VS Code),
#Vous devrez commenter cette ligne ou utiliser plt.show() à la fin.
%matplotlib inline

#============================================================================
#Etape2: Definition des fonctions de calcul de cinematique pour 2DOF
#============================================================================

def cinematique_directe_2dof(theta1, theta2, L1=10, L2=7):

    #Conversion des angles de degrés en radians
    #Les fonctions trigonométriques de numpy (cos, sin) travaillent en radians
    t1 = np.radians(theta1)
    t2 = np.radians(theta2)
    
    #Calcul des positions de chaque articulation
    #Première articulation(épaule/coude)
    x1 = L1 * np.cos(t1)
    y1 = L1 * np.sin(t1)
    
    #Deuxième articulation (coude/effecteur)
    #On ajoute la contribution du deuxième segment aux coordonnées de la première articulation
    x2 = x1 + L2 * np.cos(t1 + t2)
    y2 = y1 + L2 * np.sin(t1 + t2)
    
    return x1, y1, x2, y2


def calculer_jacobien_2dof(theta1, theta2, L1=10, L2=7):

    #Conversion des angles en radians
    t1 = np.radians(theta1)
    t2 = np.radians(theta2)
    
    #Initialisation de la matrice Jacobienne 2x2
    J = np.zeros((2, 2))
    
    #Calcul des dérivées partielles
    #Colonne1 : dérivées par rapport à θ1
    J[0, 0] = -L1 * np.sin(t1) - L2 * np.sin(t1 + t2)
    J[1, 0] = L1 * np.cos(t1) + L2 * np.cos(t1 + t2)
    
    #Colonne2 : dérivées par rapport à θ2
    J[0, 1] = -L2 * np.sin(t1 + t2)
    J[1, 1] = L2 * np.cos(t1 + t2)
    
    return J


def calculer_vitesse_effecteur_2dof(theta1, theta2, dtheta1, dtheta2, L1=10, L2=7):

    #Calcul de la matrice Jacobienne
    J = calculer_jacobien_2dof(theta1, theta2, L1, L2)
    
    #Création du vecteur des vitesses articulaires
    #conversion des degrés/s en rad/s pour le calcul
    dtheta_rad = np.radians([dtheta1, dtheta2])
    
    #Calcul de la vitesse de l'effecteur : v = J * q̇
    vitesse = J @ dtheta_rad  # @ est l'opérateur de multiplication matricielle
    
    return vitesse[0], vitesse[1]  #vx, vy


def detecter_singularites_2dof(theta1, theta2, L1=10, L2=7):

    #Calcul du Jacobien
    J = calculer_jacobien_2dof(theta1, theta2, L1, L2)
    
    #Calcul du déterminant
    det_J = np.linalg.det(J)
    
    #Conversion de θ2 en radians pour vérifier sin(θ2)
    t2 = np.radians(theta2)
    
    #Une configuration est singulière si :
    #1. Le déterminant est proche de 0
    #2. sin(θ2) est proche de 0 (bras étendu ou replié)
    is_singular = (abs(det_J) < 0.01) or (abs(np.sin(t2)) < 0.01)
    
    return is_singular, det_J


#============================================================================
#Etape3: Configuration initiale de Bras 2DOF
#============================================================================

#Définition des paramètres géométriques du bras
#Ces valeurs représentent les longueurs des segments en centimètres
L1 = 10   #Longueur du premier segment (base à coude)
L2 = 7    #Longueur du deuxième segment (coude à effecteur)

#Définition des angles initiaux en degrés
#Ces angles déterminent la configuration initiale du bras
theta1 = 45   #Angle du premier segment par rapport à l'horizontale
theta2 = 30   #Angle du deuxième segment par rapport au premier

#============================================================================
#Etape4: Calcul de la cinematique directe pour la configuration initiale
#============================================================================

#Appel de la fonction de cinématique directe avec les paramètres initiaux
x1, y1, x2, y2 = cinematique_directe_2dof(theta1, theta2, L1, L2)

#Calcul de la matrice Jacobienne pour analyser la mobilité du bras
J = calculer_jacobien_2dof(theta1, theta2, L1, L2)

#Calcul du déterminant de la matrice Jacobienne
#Pour un bras 2DOF, le déterminant indique si la configuration est singulière
det_J = np.linalg.det(J)

#Détection des singularités
is_singular, det_value = detecter_singularites_2dof(theta1, theta2, L1, L2)

#Calcul de la distance de l'effecteur par rapport à l'origine
distance = np.sqrt(x2**2 + y2**2)

#Calcul de l'angle total (orientation de l'effecteur)
angle_total = theta1 + theta2

#============================================================================
#Etape5: Affichage des resultats des calculs 
#============================================================================

print("="*70)
print("Bras Robotique 2DOF - Analyse complete de la cinematique")
print("="*70)

#Affichage des paramètres géométriques
print(f"\nParamètres GéométriquesS:")
print(f"L1 = {L1} cm (base à coude)")
print(f"L2 = {L2} cm (coude à effecteur)")  
print(f"Longueur totale: {L1 + L2} cm")

#Affichage de la configuration des angles
print(f"\nConfiguration des Angles:")
print(f"θ1 = {theta1}° (rotation de base)")
print(f"θ2 = {theta2}° (rotation du coude)")
print(f"Angle total (θ1 + θ2) = {angle_total}°")

#Affichage des positions calculées par la cinématique directe
print(f"\nCinématique Directe - Positions Calculées:")
print(f"Articulation 1 (coude):   ({x1:.2f}, {y1:.2f}) cm")
print(f"Effecteur final:          ({x2:.2f}, {y2:.2f}) cm")

#Affichage des analyses géométriques
print(f"\nAnalyses Géométriques:")
print(f"Distance depuis l'origine: {distance:.2f} cm")
print(f"Distance maximale possible: {L1 + L2} cm")
print(f"Pourcentage d'extension: {(distance/(L1+L2))*100:.1f}%")
print(f"Rayon de l'effecteur: {distance:.2f} cm")
print(f"Angle polaire: {np.degrees(np.arctan2(y2, x2)):.1f}°")

#Affichage de la matrice Jacobienne
print(f"\nMatrice Jacobienne(2x2):")
print(f"  J = [ [{J[0,0]:.3f}  {J[0,1]:.3f}]")
print(f"        [{J[1,0]:.3f}  {J[1,1]:.3f}] ]")

#Analyse du Jacobien et détection de singularités
print(f"\nAnalyse du Jacobien:")
print(f"Déterminant de J: {det_J:.3f}")
print(f"Valeur absolue du déterminant: {abs(det_J):.3f}")

if is_singular:
    print("Configuration Singulière - Perte de mobilité.")
    print("Le bras ne peut pas se déplacer dans toutes les directions.")
    
    #Identification du type de singularité
    t2_rad = np.radians(theta2)
    if abs(np.sin(t2_rad)) < 0.01:
        if abs(theta2) < 1:  # Bras étendu
            print("Type: Bras complètement étendu (θ2 ≈ 0°)")
        else:  #Bras replié
            print("Type: Bras complètement replié (θ2 ≈ 180° ou -180°)")
else:
    print("Configuration non singulière")
    print("Le bras peut se déplacer dans toutes les directions du plan.")

print(f"\nInvesibilité:")
if abs(det_J) > 0.01:
    print(f"Le Jacobien est inversible")
    print(f"La cinématique inverse existe pour cette configuration")
else:
    print(f"Le Jacobien n'est pas inversible")
    print(f"La cinématique inverse n'existe pas pour cette configuration")

#============================================================================
#Etape6: Visualisation statique avec informations de cinematique
#============================================================================

#Création d'une figure avec 6 sous-graphiques(2 lignes × 3 colonnes)
fig = plt.figure(figsize=(16, 8))

#----------------------------------------------------------------------------
#Sous-figure 1 : Vue du bras avec annotations(occupant les positions(1,1) et (2,1))
#----------------------------------------------------------------------------
ax1 = plt.subplot(2, 3, (1, 4))  #(1,4) signifie lignes 1-2, colonne 1

#Tracé du bras : ligne reliant les points (0,0) → (x1,y1) → (x2,y2)
ax1.plot([0, x1, x2], [0, y1, y2], marker='o', linewidth=3, markersize=10, color='blue')

#Ajout des points de base et d'effecteur avec des couleurs différentes
ax1.scatter([0], [0], color='red', s=150, label='Base', zorder=5)
ax1.scatter([x2], [y2], color='green', s=150, label='Effecteur', zorder=5)

#Ajout des annotations de longueurs sur le graphique
ax1.annotate(f'L1={L1}', xy=(x1/2, y1/2), xytext=(x1/2-1, y1/2+1),
fontsize=10, color='darkblue', fontweight='bold')
ax1.annotate(f'L2={L2}', xy=((x1+x2)/2, (y1+y2)/2), 
             xytext=((x1+x2)/2-1, (y1+y2)/2+1),
             fontsize=10, color='darkblue', fontweight='bold')

#Ajout des annotations d'angles
#Angle θ1 : arc de cercle à l'origine
angle_theta1 = np.linspace(0, np.radians(theta1), 50)
radius = 2
ax1.plot(radius * np.cos(angle_theta1), radius * np.sin(angle_theta1), 
         'r-', linewidth=2)
ax1.annotate(f'θ1={theta1}°', xy=(radius*np.cos(np.radians(theta1/2)), 
                                   radius*np.sin(np.radians(theta1/2))),
             xytext=(radius*np.cos(np.radians(theta1/2))+1, 
                     radius*np.sin(np.radians(theta1/2))+1),
             color='red', fontweight='bold')

#Angle θ2 : arc de cercle au niveau de la première articulation
angle_theta2 = np.linspace(np.radians(theta1), np.radians(theta1+theta2), 50)
radius2 = 2
x_center, y_center = x1, y1
ax1.plot(x_center + radius2 * np.cos(angle_theta2), 
         y_center + radius2 * np.sin(angle_theta2), 
         'orange', linewidth=2)
ax1.annotate(f'θ2={theta2}°', 
             xy=(x_center + radius2*np.cos(np.radians(theta1 + theta2/2)), 
                  y_center + radius2*np.sin(np.radians(theta1 + theta2/2))),
             xytext=(x_center + radius2*np.cos(np.radians(theta1 + theta2/2)) + 1, 
                     y_center + radius2*np.sin(np.radians(theta1 + theta2/2)) + 1),
             color='orange', fontweight='bold')

#Configuration des axes et ajout de la grille
ax1.set_xlim(-20, 20)
ax1.set_ylim(-20, 20)
ax1.set_aspect('equal')  #Garde les mêmes échelles sur X et Y
ax1.grid(True, alpha=0.3)
ax1.set_title("Représentation géométrique du bras 2DOF", fontsize=12, fontweight='bold')
ax1.set_xlabel("Position X (cm)")
ax1.set_ylabel("Position Y (cm)")
ax1.legend()

#----------------------------------------------------------------------------
#Sous-figure 2 : Diagramme des angles en barres
#----------------------------------------------------------------------------
ax2 = plt.subplot(2, 3, 2)  #Ligne 1, colonne 2
angles = [theta1, theta2, angle_total]
labels = ['θ1', 'θ2', 'θ1+θ2']
colors = ['red', 'orange', 'green']
bars = ax2.bar(labels, angles, color=colors, alpha=0.7)
ax2.set_ylim(0, 180)
ax2.set_title("Configuration des angles", fontsize=12, fontweight='bold')
ax2.set_ylabel("Angle (degrés)")
ax2.grid(True, alpha=0.3, axis='y')

#Ajout des valeurs numériques sur les barres
for bar, angle in zip(bars, angles):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{angle}°', ha='center', va='bottom', fontweight='bold')

#----------------------------------------------------------------------------
#Sous-figure 3 : Affichage de la matrice Jacobienne et de son déterminant
#----------------------------------------------------------------------------
ax3 = plt.subplot(2, 3, 3)  #Ligne 1, colonne 3
ax3.axis('off')  #Désactive les axes pour afficher du texte

#Création du texte pour le Jacobien
jacobien_text = f"""
Matrice Jacobienne(2x2)

J = [{J[0,0]:6.3f}  {J[0,1]:6.3f}]
    [{J[1,0]:6.3f}  {J[1,1]:6.3f}]

Analyse:
- det(J) = {det_J:6.3f}
- |det(J)| = {abs(det_J):6.3f}
- Inversible: {'Oui' if abs(det_J) > 0.01 else 'Non'}

Singularite:
{'Configuration Singuliere' if is_singular else 'Configuration Normal'}

Equation de Determinant:
det(J) = L1·L2·sin(θ2)
       = {L1}·{L2}·sin({theta2}°)
       = {L1*L2}·{np.sin(np.radians(theta2)):.3f}
       = {det_J:.3f}
"""
ax3.text(0.1, 0.5, jacobien_text, fontfamily='monospace', fontsize=10,
         verticalalignment='center',
         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))
ax3.set_title("Analyse cinématique", fontsize=12, fontweight='bold')

#----------------------------------------------------------------------------
#Sous-figure 4 : Informations numériques détaillées
#----------------------------------------------------------------------------
ax4 = plt.subplot(2, 3, 5)  #Ligne 2, colonne 2
ax4.axis('off')

#Création du texte avec les informations numériques
info_text = f"""
Positions Exactes:

Base: (0.000, 0.000) cm

Articulation 1 (coude):
  x1 = {x1:7.3f} cm
  y1 = {y1:7.3f} cm

Effecteur final:
  x2 = {x2:7.3f} cm  
  y2 = {y2:7.3f} cm

Caracteristiques Geometriques:
Distance: {distance:7.3f} cm
Extension: {(distance/(L1+L2))*100:6.1f} %
Longueur tot: {L1+L2:7.1f} cm
Angle polaire: {np.degrees(np.arctan2(y2, x2)):6.1f}°
"""
ax4.text(0.1, 0.5, info_text, fontfamily='monospace', fontsize=10,
         verticalalignment='center',
         bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
ax4.set_title("Données numériques", fontsize=12, fontweight='bold')

#----------------------------------------------------------------------------
#Sous-figure 5 : Espace de travail du bras 2DOF
#----------------------------------------------------------------------------
ax5 = plt.subplot(2, 3, 6)  #Ligne 2, colonne 3

#Génération de l'espace de travail : ensemble de tous les points atteignables
#par l'effecteur en faisant varier tous les angles possibles
theta1_range = np.arange(0, 360, 10)    #θ1 de 0 à 360° par pas de 10°
theta2_range = np.arange(-180, 180, 5)  #θ2 de -180 à 180° par pas de 5°
points_2dof = []

#Double boucle pour explorer toutes les combinaisons d'angles
for t1 in theta1_range:
    for t2 in theta2_range:
        #Calcul de la position de l'effecteur pour cette combinaison
        _, _, x2_temp, y2_temp = cinematique_directe_2dof(t1, t2, L1, L2)
        points_2dof.append([x2_temp, y2_temp])

#Conversion en tableau numpy pour le tracé
points_2dof = np.array(points_2dof)

#Tracé de l'espace de travail
#Les points sont colorés selon leur distance à l'origine
distances = np.sqrt(points_2dof[:, 0]**2 + points_2dof[:, 1]**2)
scatter = ax5.scatter(points_2dof[:, 0], points_2dof[:, 1], 
                      c=distances, cmap='viridis', s=2, alpha=0.5)

#Ajout d'un marqueur pour la position actuelle
ax5.scatter(x2, y2, s=150, color='red', marker='*', 
            label=f'Position actuelle\n({x2:.1f}, {y2:.1f})', zorder=5)

#Ajout d'un cercle montrant la limite de l'espace de travail
#Pour un bras 2DOF, l'espace de travail est une couronne circulaire
rayon_max = L1 + L2
rayon_min = abs(L1 - L2)
cercle_max = plt.Circle((0, 0), rayon_max, color='red', fill=False, 
                        linestyle='--', linewidth=2, alpha=0.7, label='Limite max')
cercle_min = plt.Circle((0, 0), rayon_min, color='blue', fill=False, 
                        linestyle='--', linewidth=2, alpha=0.7, label='Limite min')
ax5.add_patch(cercle_max)
ax5.add_patch(cercle_min)

#Configuration des axes
ax5.set_xlim(-20, 20)
ax5.set_ylim(-20, 20)
ax5.set_aspect('equal')
ax5.grid(True, alpha=0.3)
ax5.set_title("Espace de travail 2DOF", fontsize=12, fontweight='bold')
ax5.set_xlabel("X (cm)")
ax5.legend(loc='upper right')

#Ajout d'une barre de couleur pour les distances
cbar = plt.colorbar(scatter, ax=ax5)
cbar.set_label('Distance depuis l\'origine (cm)')

#Ajustement automatique de l'espacement entre les sous-graphiques
plt.tight_layout()
plt.show()

#============================================================================
#Etape8: Animation avec calculs en temps reel pour 2DOF
#============================================================================

print("\n" + "="*70)
print("Animation 2DOF avec calculs de cinematique en temps reel")
print("="*70)

#Création d'une nouvelle figure pour l'animation avec 3 sous-graphiques côte à côte
fig_anim, axes_anim = plt.subplots(1, 3, figsize=(18, 5))

#----------------------------------------------------------------------------
#Configuration du premier axe : vue du bras
#----------------------------------------------------------------------------
axes_anim[0].set_xlim(-20, 20)
axes_anim[0].set_ylim(-20, 20)
axes_anim[0].set_aspect('equal')
axes_anim[0].grid(True, alpha=0.3)
axes_anim[0].set_title("Animation du bras 2DOF", fontsize=12, fontweight='bold')
axes_anim[0].set_xlabel("Position X (cm)")
axes_anim[0].set_ylabel("Position Y (cm)")

#----------------------------------------------------------------------------
#Configuration du deuxième axe : évolution des angles dans le temps
#----------------------------------------------------------------------------
axes_anim[1].set_xlim(0, 360)
axes_anim[1].set_ylim(-180, 180)
axes_anim[1].grid(True, alpha=0.3)
axes_anim[1].set_title("Évolution des angles", fontsize=12, fontweight='bold')
axes_anim[1].set_xlabel("Frame")
axes_anim[1].set_ylabel("Angle (degrés)")
axes_anim[1].axhline(y=0, color='k', linestyle='-', alpha=0.3)

#----------------------------------------------------------------------------
#Configuration du troisième axe : informations textuelles
#----------------------------------------------------------------------------
axes_anim[2].axis('off')  #Désactive les axes pour afficher du texte
#Création d'un objet texte qui sera mis à jour à chaque frame
info_text_obj = axes_anim[2].text(0.05, 0.5, '', fontfamily='monospace', fontsize=9,
                                 verticalalignment='center',
                                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))

#----------------------------------------------------------------------------
#Création des objets graphiques qui seront animés
#----------------------------------------------------------------------------
#Ligne représentant le bras
line_anim, = axes_anim[0].plot([], [], marker='o', linewidth=2, markersize=8, color='blue')
#Trajectoire de l'effecteur (ligne pointillée verte)
trajectory_anim, = axes_anim[0].plot([], [], 'g--', alpha=0.5, linewidth=1)
#Point représentant l'effecteur
effecteur_dot_anim, = axes_anim[0].plot([], [], 'go', markersize=10, alpha=0.7)

#Lignes pour l'historique des angles
angle1_line_anim, = axes_anim[1].plot([], [], 'r-', label='θ1', linewidth=2)
angle2_line_anim, = axes_anim[1].plot([], [], 'orange', label='θ2', linewidth=2)
angle_total_line_anim, = axes_anim[1].plot([], [], 'g-', label='θ1+θ2', linewidth=2, alpha=0.5)
axes_anim[1].legend()  #Ajout de la légende pour identifier les courbes

#----------------------------------------------------------------------------
#Variables pour stocker l'historique des positions et angles
#----------------------------------------------------------------------------
x_history_anim = []  #Historique des positions X de l'effecteur
y_history_anim = []  #Historique des positions Y de l'effecteur
theta1_history_anim = []  #Historique de l'angle θ1
theta2_history_anim = []  #Historique de l'angle θ2
theta_total_history_anim = []  #Historique de l'angle total
frame_history_anim = []   #Historique des numéros de frame

#----------------------------------------------------------------------------
#Définition de la fonction de mise à jour appelée à chaque frame
#----------------------------------------------------------------------------
def update_animation(frame):

    #------------------------------------------------------------------------
    #1.Calcul des nouveaux angles pour cette frame
    #------------------------------------------------------------------------
    #Chaque angle suit une loi de variation différente pour créer
    #Un mouvement intéressant et montrer différentes configurations
    
    #θ1 varie sinusoïdalement avec une amplitude de 180° et une fréquence de 0.5
    #La formule : 90 * sin(0.5*frame) + 90 donne des valeurs entre 0° et 180°
    theta1_current = 90 * np.sin(np.radians(frame * 0.5)) + 90
    
    #θ2 varie sinusoïdalement avec une amplitude de 90° et une fréquence de 1.0
    #La formule : 45 * sin(1.0*frame) + 45 donne des valeurs entre 0° et 90°
    theta2_current = 45 * np.sin(np.radians(frame * 1.0)) + 45
    
    #Calcul de l'angle total
    theta_total_current = theta1_current + theta2_current
    
    #------------------------------------------------------------------------
    #2.Calcul des vitesses articulaires(derivees des angles)
    #------------------------------------------------------------------------
    #Dérivée de A*sin(ωt) + C : A*ω*cos(ωt)
    dtheta1 = 90 * 0.5 * np.cos(np.radians(frame * 0.5)) * (np.pi/180)
    dtheta2 = 45 * 1.0 * np.cos(np.radians(frame * 1.0)) * (np.pi/180)
    
    #------------------------------------------------------------------------
    #3.Calculs de cinematique directe 
    #------------------------------------------------------------------------
    x1_curr, y1_curr, x2_curr, y2_curr = cinematique_directe_2dof(
        theta1_current, theta2_current, L1, L2)
    
    #------------------------------------------------------------------------
    #4.Calculs de cinematique differentielle(Jacobien)
    #------------------------------------------------------------------------
    J_curr = calculer_jacobien_2dof(theta1_current, theta2_current, L1, L2)
    det_J_curr = np.linalg.det(J_curr)
    
    #------------------------------------------------------------------------
    #5.Detection des singiularites
    #------------------------------------------------------------------------
    is_singular_curr, _ = detecter_singularites_2dof(theta1_current, theta2_current, L1, L2)
    
    #------------------------------------------------------------------------
    #6.Calcul de la vitessse de l'effecteur
    #------------------------------------------------------------------------
    vx, vy = calculer_vitesse_effecteur_2dof(theta1_current, theta2_current,
                                           dtheta1*180/np.pi, dtheta2*180/np.pi, L1, L2)
    vitesse_norme = np.sqrt(vx**2 + vy**2)
    
    #------------------------------------------------------------------------
    #7.Calcul de la distance depuis l'origine
    #------------------------------------------------------------------------
    distance_curr = np.sqrt(x2_curr**2 + y2_curr**2)
    
    #------------------------------------------------------------------------
    #8.Mise a jour du graphique du bras
    #------------------------------------------------------------------------
    line_anim.set_data([0, x1_curr, x2_curr], [0, y1_curr, y2_curr])
    
    #------------------------------------------------------------------------
    #9.Mise a jour de la trajectoire
    #------------------------------------------------------------------------
    x_history_anim.append(x2_curr)
    y_history_anim.append(y2_curr)
    #Limite la longueur de l'historique pour des raisons de performance
    if len(x_history_anim) > 100:
        x_history_anim.pop(0)
        y_history_anim.pop(0)
    trajectory_anim.set_data(x_history_anim, y_history_anim)
    effecteur_dot_anim.set_data([x2_curr], [y2_curr])
    
    #------------------------------------------------------------------------
    #10.Mise a jour de graphique des angles
    #------------------------------------------------------------------------
    frame_history_anim.append(frame)
    theta1_history_anim.append(theta1_current)
    theta2_history_anim.append(theta2_current)
    theta_total_history_anim.append(theta_total_current)
    #Limite la longueur de l'historique
    if len(frame_history_anim) > 200:
        frame_history_anim.pop(0)
        theta1_history_anim.pop(0)
        theta2_history_anim.pop(0)
        theta_total_history_anim.pop(0)
    
    angle1_line_anim.set_data(frame_history_anim, theta1_history_anim)
    angle2_line_anim.set_data(frame_history_anim, theta2_history_anim)
    angle_total_line_anim.set_data(frame_history_anim, theta_total_history_anim)
    
    #------------------------------------------------------------------------
    #11.Mise a jour des informations textuelles
    #------------------------------------------------------------------------
    info_text = f"""
Frame: {frame:3.0f}

Angles:
θ1 = {theta1_current:6.1f}°
θ2 = {theta2_current:6.1f}°
θ1+θ2 = {theta_total_current:6.1f}°

Positions:
Base: (0.000, 0.000) cm
Coude: ({x1_curr:6.2f}, {y1_curr:6.2f}) cm
Effecteur: ({x2_curr:6.2f}, {y2_curr:6.2f}) cm

Jacobien(2×2):
{J_curr[0,0]:6.2f} {J_curr[0,1]:6.2f}
{J_curr[1,0]:6.2f} {J_curr[1,1]:6.2f}

Analyse:
det(J) = {det_J_curr:6.3f}
Distance = {distance_curr:6.2f} cm
vx = {vx:6.2f} cm/s
vy = {vy:6.2f} cm/s
|v| = {vitesse_norme:6.2f} cm/s

{'Singulier (sin(θ2)≈0)' if is_singular_curr else 'Normal'}
"""
    info_text_obj.set_text(info_text)
    
    #------------------------------------------------------------------------
    #12.Mise a jour de titre
    #------------------------------------------------------------------------
    axes_anim[0].set_title(f"Bras 2DOF - Frame {frame:.0f}", fontsize=12, fontweight='bold')
    
    #------------------------------------------------------------------------
    #13.Retour des objets a mettre a jour
    #------------------------------------------------------------------------
    return (line_anim, trajectory_anim, effecteur_dot_anim, 
            angle1_line_anim, angle2_line_anim, angle_total_line_anim, info_text_obj)

#----------------------------------------------------------------------------
#Création de l'objet animation
#----------------------------------------------------------------------------
ani_advanced = FuncAnimation(
    fig=fig_anim,           #Figure à animer
    func=update_animation,  #Fonction de mise à jour
    frames=np.arange(0, 720, 1.5),  #Séquence de frames : de 0 à 720 par pas de 1.5
    interval=30,            #Intervalle entre les frames en ms (30 ms ≈ 33 fps)
    blit=True,              #Optimisation : ne redessine que ce qui a changé
    repeat=True             #L'animation se répète en boucle
)

#----------------------------------------------------------------------------
#Affichage de l'animation dans Jupyter Notebook
#----------------------------------------------------------------------------
print("\nLancement de l'animation avec calculs en temps réel...")
print("-Cinématique directe (positions)")
print("-Matrice Jacobienne 2×2")
print("-Calcul du déterminant")
print("-Détection de singularités (sin(θ2) ≈ 0)")
print("-Vitesse de l'effecteur")
print("-Analyse de mobilité\n")

#Conversion de l'animation en HTML/JavaScript pour l'affichage dans Jupyter
display(HTML(ani_advanced.to_jshtml()))

#============================================================================
#Etape9 : Analyses supplementaires de cinematique pour 2DOF
#============================================================================

print("\n" + "="*70)
print("Analyses supplementaires de cinematique - 2DOF")
print("="*70)

#----------------------------------------------------------------------------
#1.Calcul de l'espace de travail complet
#----------------------------------------------------------------------------
print("\n1.Espace de travaille de bras 2DOF:")

#Pour un bras 2DOF, l'espace de travail est une couronne circulaire (anneau)
#Rayon intérieur : |L1 - L2| (bras replié)
#Rayon extérieur : L1 + L2 (bras étendu)
rayon_interieur = abs(L1 - L2)
rayon_exterieur = L1 + L2

print(f"Rayon intérieur: |L1 - L2| = |{L1} - {L2}| = {rayon_interieur:.2f} cm")
print(f"Rayon extérieur: L1 + L2 = {L1} + {L2} = {rayon_exterieur:.2f} cm")
print(f"Zone annulaire: {rayon_interieur:.2f} cm ≤ r ≤ {rayon_exterieur:.2f} cm")
print(f"Surface atteignable: π(R² - r²) = {np.pi*(rayon_exterieur**2 - rayon_interieur**2):.2f} cm²")

#----------------------------------------------------------------------------
#2.Analyse des configurations singulières
#----------------------------------------------------------------------------
print("\n2.Analyse des configurations singulières:")

print("Pour un bras 2DOF, le déterminant du Jacobien est:")
print("det(J) = L1 * L2 * sin(θ2)")
print(f"Avec L1 = {L1}, L2 = {L2}")
print("Donc det(J) = 0 lorsque sin(θ2) = 0")
print("Soit lorsque θ2 = 0° ou θ2 = ±180°")

#Exploration des configurations singulières
configurations_singulieres = [
    (0, 0, "Bras complètement étendu"),
    (0, 180, "Bras replié vers l'avant"),
    (0, -180, "Bras replié vers l'arrière"),
    (90, 0, "Bras en L étendu"),
    (180, 0, "Bras étendu vers l'arrière")
]

print("\n Exemples de configurations singulières:")
for t1, t2, description in configurations_singulieres:
    is_singular, det_val = detecter_singularites_2dof(t1, t2, L1, L2)
    print(f" -{description:25s}: θ1={t1:3d}°, θ2={t2:4d}°, det(J)={det_val:7.3f}, {'SINGULIER' if is_singular else 'normal'}")

#----------------------------------------------------------------------------
#3.Analyse de dextérité (conditionnement du Jacobien)
#----------------------------------------------------------------------------
print("\n3.Analyse de dextérité (conditionnement du Jacobien):")

#Le conditionnement mesure la sensibilité du système
#Pour une matrice 2x2, conditionnement = |λ_max| / |λ_min|
#où λ sont les valeurs propres

configurations_test = [
    (45, 90, "Configuration optimale"),
    (0, 90, "Bras vertical"),
    (45, 0, "Bras étendu (singulier)"),
    (45, 180, "Bras replié (singulier)"),
    (45, 45, "Configuration standard")
]

print("\nConditionnement pour différentes configurations:")
print("(Un conditionnement proche de 1 est optimal)")
print("(Un conditionnement élevé indique une singularité)")

for t1, t2, description in configurations_test:
    J_test = calculer_jacobien_2dof(t1, t2, L1, L2)
    #Calcul des valeurs singulières
    U, s, Vt = np.linalg.svd(J_test)
    if len(s) >= 2 and s[1] > 1e-10:
        conditionnement = s[0] / s[1]
    else:
        conditionnement = float('inf')
    
    #Calcul du déterminant pour référence
    det_val = np.linalg.det(J_test)
    
    print(f" -{description:25s}: θ1={t1:3d}°, θ2={t2:3d}°, cond={conditionnement:7.2f}, det={det_val:7.3f}")

#----------------------------------------------------------------------------
#4.Calcul de la cinématique inverse (géométrique)
#----------------------------------------------------------------------------
print("\n4.Cinématique inverse:")

#Pour un bras 2DOF, la cinématique inverse peut être résolue géométriquement
#Étant donné une position cible (x, y), on peut calculer les angles θ1 et θ2

def cinematique_inverse_2dof(x, y, L1=10, L2=7):

    #Distance à l'origine
    d = np.sqrt(x**2 + y**2)
    
    #Vérification si la position est atteignable
    if d > (L1 + L2) or d < abs(L1 - L2):
        return None, None  #Position hors de l'espace de travail
    
    #Calcul de cos(θ2) via la loi des cosinus
    cos_theta2 = (d**2 - L1**2 - L2**2) / (2 * L1 * L2)
    
    #S'assurer que cos_theta2 est dans [-1, 1] (à cause des erreurs d'arrondi)
    cos_theta2 = max(-1, min(1, cos_theta2))
    
    #Deux solutions pour θ2 (coude haut et coude bas)
    theta2_1 = np.degrees(np.arccos(cos_theta2))  #Coude haut (θ2 positif)
    theta2_2 = -theta2_1  #Coude bas (θ2 négatif)
    
    #Calcul de α (angle de la position cible)
    alpha = np.degrees(np.arctan2(y, x))
    
    #Calcul de β via la loi des cosinus
    cos_beta = (L1**2 + d**2 - L2**2) / (2 * L1 * d)
    cos_beta = max(-1, min(1, cos_beta))
    beta = np.degrees(np.arccos(cos_beta))
    
    #Deux solutions pour θ1 correspondant aux deux solutions pour θ2
    theta1_1 = alpha - beta  #Pour θ2 positif
    theta1_2 = alpha + beta  #Pour θ2 négatif
    
    return (theta1_1, theta2_1), (theta1_2, theta2_2)

#Exemple de calcul de cinématique inverse
print("\nExemple de cinématique inverse:")
x_target, y_target = 12, 5
print(f"Position cible: ({x_target}, {y_target})")

solutions = cinematique_inverse_2dof(x_target, y_target, L1, L2)

if solutions[0] is None:
    print("Position hors de l'espace de travail!")
else:
    sol1, sol2 = solutions
    print(f"Solution 1 (coude haut): θ1={sol1[0]:.1f}°, θ2={sol1[1]:.1f}°")
    print(f"Solution 2 (coude bas):  θ1={sol2[0]:.1f}°, θ2={sol2[1]:.1f}°")
    
    #Vérification des solutions
    for i, (theta1_sol, theta2_sol) in enumerate([sol1, sol2], 1):
        x1_check, y1_check, x2_check, y2_check = cinematique_directe_2dof(theta1_sol, theta2_sol, L1, L2)
        erreur = np.sqrt((x2_check - x_target)**2 + (y2_check - y_target)**2)
        print(f"Vérification sol {i}: erreur = {erreur:.3f} cm")

#----------------------------------------------------------------------------
#5.Comparaison avec le bras 3DOF
#----------------------------------------------------------------------------
print("\n5.Comparaison: 2DOF vs 3DOF:")

print("Caracteristiques de 2DOF:")
print("- Degrés de liberté: 2")
print("- Espace de travail: Couronne circulaire (anneau)")
print("- Singularités: Lorsque sin(θ2) = 0")
print("- Cinématique inverse: 2 solutions (coude haut/coude bas)")
print("- Applications: Manipulateurs simples, pick-and-place")

print("\nAvantages de 2DOF:")
print("- Simplicité de conception et de contrôle")
print("- Coût réduit")
print("- Calculs plus simples")
print("- Moins de singularités que le 3DOF")

print("\nLimitaions de 2DOF:")
print("- Impossible de contrôler l'orientation de l'effecteur")
print("- Espace de travail limité (pas de points à l'intérieur du rayon minimal)")
print("- Moins de dextérité que le 3DOF")

#============================================================================
#Etape10:Fin de programme
#============================================================================
print("\n" + "="*70)
print("Analyse Terminee- BRAS 2DOF.")
print("="*70)